<a href="https://colab.research.google.com/github/takatakamanbou/Data/blob/main/Data2024_ageestimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 回帰分析の応用: 顔画像からの年齡推定



## はじめに



この授業で学んだ回帰分析は，説明変数 $x$ と被説明変数 $y$ の値のペアから成るデータの集まり $\{(x_n, y_n)\}$ （$n=1, 2, \ldots, N$）が与えられたときに，これらによく当てはまる直線 $y = ax+b$ を求める，つまりこの式のパラメータ $a, b$ を求めるというデータ分析手法でした．

この授業の範囲では説明変数は一つでしたが，二つ以上の場合に拡張することができます．その場合，データの集まりを $\{ (x_{n,1}, x_{n,2}, \ldots, x_{n,D}, y_{n}) \}$ （$n=1, 2, \ldots, N$）として，$D$ 個の説明変数 $x_{1}, x_{2}, \ldots, x_{D}$ から被説明変数 $y$ を予測する式を

$$
y = w_0 + w_1x_1 + w_2x_2 + \cdots + w_Dx_D \qquad (1)
$$

とおけば，直線を当てはめる場合と同じように，データに平面（$D$次元超平面）を当てはめる最小二乗法を導くことができます．
このような複数の説明変数を持つデータに対する回帰分析を，「重回帰分析」といいます（説明変数がひとつだけの場合を「単回帰分析」ということもあります）．「重回帰分析」は，この授業ではなく2年次の「多変量解析及び演習」で学ぶものですが，ここでは，回帰分析の応用例として，顔画像から年齢を推定する実験をやってみましょう．


推定の精度はあまり高くありませんが，回帰分析のようなデータ分析の手法がこんなところでも使えるよ，ということが伝わればいいな．
ちなみに，もっと推定精度を高めたければ，もっと複雑な，機械学習・AIの手法を使うことになります．その辺は3年次の「機械学習I,II」で学べるかも．

## 実験方法の概要

人の顔が中央に大きく写った幅 $64$ 画素高さ $64$ 画素の $3$ チャンネルカラー画像を考えます．
これらの画像に写った人物の年齡を表す整数値も与えられているものとします．
このとき，一つの画像の画素値は $64\times 64\times 3 = 12288$ 個あります．この画像から年齢を推定する問題は，$D = 12288$ 個の画素値を説明変数とし，年齢を被説明変数とする重回帰分析の問題とみなせます．

というわけで，実際の人の顔とその年齢のデータを $38138$ 件集めたデータに重回帰分析を適用して，式$(1)$のパラメータ $w_0, w_1, \ldots, w_{12288}$ を求めたものを用意しました（注）．適当な画像の画素値 $x_1, x_2, \ldots, x_{12288}$ に対して式$(1)$の値を計算することで，その画像に写ったひとの年齢を推定させてみましょう．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: ここでは説明を簡単にするために少々嘘をついています．本当の方法については，この notebook 末尾の「補足」で説明しています．
</span>



## 準備

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

import cv2  # Python による「コンピュータビジョン(Computer Vision)」のためのライブラリ OpenCV のパッケージをインポート

# 顔検出器の準備
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

次のコードセルでは，あとで使う関数を定義しています．このセルを実行しただけでは何も起こりませんが，あらかじめ実行して関数を定義しておかないと後のセルが動きません．

In [ ]:
# Colab へのファイルアップロードを実行する関数
#
def uploadToColab():
    try:
        from google.colab import files
        rv = files.upload()
    except:
        print('このコードは Colab 以外の環境では実行できないよ．')


# OpenCVの形式の画像を表示する関数
#
def imshow(img):
    try:
        from google.colab.patches import cv2_imshow
        cv2_imshow(img) # Colab上で実行している場合
    except:
        cv2.imshow(img)  # それ以外の場合


# 与えられた画像の中から顔を検出する
#
def faceDetector(img, faceCascade, maxSize=400):

    h, w = img.shape[:2]

    # (短辺の長さ) <= maxSize にリサイズ
    if min(w, h) > maxSize:
        if w <= h:
            w2, h2 = maxSize, h*maxSize//w
        else:
            w2, h2 = w*maxSize//h, maxSize
        imgDisp = cv2.resize(img, (w2, h2))
    else:
        imgDisp = np.copy(img)

    # 顔検出を実行
    imgGray = cv2.cvtColor(imgDisp, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(imgGray, 1.1, 4)
    nFace = len(faces)

    # 検出した顔のうち最初のひとりを選んで切り取り
    if nFace > 0:
        print(faces)
        x, y, ww, hh = faces[0]
        imgFace = np.copy(imgDisp[y:y+hh, x:x+ww, :])
        cv2.rectangle(imgDisp, (x, y), (x+ww, y+hh), color=(0, 255, 0), thickness=2)
        return nFace, imgDisp, imgFace
    else:
        return nFace, imgDisp, None


用意しておいた $w_0, w_1, \ldots, w_D$ などの値を読み込みます．

In [ ]:
# takataka のウェブサイトからパラメータを格納したファイルを Colab へダウンロード
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/ageestimation.npz

import os
path = 'ageestimation.npz'
if os.path.exists(path):
    # パラメータを読み込む
    params = np.load(path)
    print(f'ファイル {path} を読み込みました')
    Xm  = params['Xm']  # 平均ベクトル
    eve = params['eve'] # 主成分分析で得られた固有ベクトル
    w   = params['w']   # 重回帰分析で得られたパラメータ
    print(Xm.shape, eve.shape, w.shape)
else:
    print(f'ファイル {path} の読み込みに失敗したようです．再実行してみてください')

## 実験

(1) 画像を Colab へアップロードします．次のような画像にしてください

- ひとりのひとのほぼ正面を向いた顔全体が写ってる．複数人を検出した場合，ひとりだけ抜き出します（あとで条件に合わない画像をわざと与えて実験してみるのもよいでしょう）．
- 画像全体に顔がドアップで写っているような場合はうまく顔検出ができないかもしれません．顔がもう少し小さく写ってる画像を探してみましょう．
- ファイル名に空白やマルチバイト文字（日本語など）が含まれているとうまく動作しない場合があるので，自分のPCの方で適当な名前に変えておく（拡張子は変えてはいけない）．


In [ ]:
# Colab へファイルをアップロード
uploadToColab()

# ls コマンドでファイルを一覧
! ls

上のセルを実行してファイルをアップロードできたら，次のセルのファイル名の部分にその名前を指定して，読み込ませましょう．

In [ ]:
# 画像を読み込む．`hoge.jpg` を自分がアップロードしたファイルの名前に修正
img = cv2.imread('hoge.jpg')

(2) 顔検出を実行して顔領域を切り取った画像を作る

In [ ]:
numFace, imgDisp, imgFace = faceDetector(img, faceCascade, maxSize=400)
if numFace > 0:
    imshow(imgDisp)
    imshow(imgFace)
else:
    imshow(imgDisp)
    print('顔を検出できませんでした')

(3) 顔を所定の大きさにリサイズしてから次元削減し，年齡推定を実行．

In [ ]:
# 顔画像を 64 x 64 にリサイズして 12288次元ベクトルに
xvec = cv2.resize(imgFace, (64, 64)).reshape(-1)
print(xvec.shape)

# PCAを利用した次元削減
yvec = (xvec - Xm) @ eve
print(yvec.shape)

# 予測値の計算
yvec1 = np.concatenate(([1], yvec)) # 先頭に 1 をくっつける
z = w @ yvec1

# 結果の表示
print(f'このひとの年齡の推定値は {z:.1f} です')

## 補足

### 実験方法についての補足

「方法の概要」では，$D = 12288$ 個の画素値を説明変数とした重回帰分析を行っているように説明しています．しかし実際には，このような画素値そのままのデータは次元数が大きすぎて扱いが難しいので，「主成分分析」という手法を用いて，データの次元数をいったん $D = 2151$ まで削減してから重回帰分析を適用しています．「主成分分析」についても，「多変量解析及び演習」で学ぶことができます．

### データ分析や多変量解析と線形代数の関係？

「データ分析」や「多変量解析及び演習」の授業で学ぶ内容は，大学初年次で学ぶ微積分や線形代数，確率統計の初歩と密接な関わりがあります．ここでは，特に線形代数との関わりについて，ちょびっとだけ説明します．

データ $\{(x_n, y_n)\}$（$n = 1, 2, \ldots, N$）に $y = w_0 + w_1x$ という直線を当てはめることを考えます．このとき，

$$
X = \begin{pmatrix}
1 & 1 & \cdots & 1 \\
x_{1} & x_{2} & \cdots & x_{N}
\end{pmatrix}\quad Y = \begin{pmatrix} y_1 & y_2 & \cdots & y_N\end{pmatrix}
$$

という $2\times N$ 行列 $X$ と $1\times D$ 行列 $Y$ に対して，

$$
XX^{\top} =
\begin{pmatrix}
\displaystyle\sum_{n=1}^{N}1 & \displaystyle\sum_{n=1}^{N}x_n\\
\displaystyle\sum_{n=1}^{N}x_n & \displaystyle\sum_{n=1}^{N}x_n^2\\
\end{pmatrix} \quad
XY^{\top} =
\begin{pmatrix}
\displaystyle\sum_{n=1}^{N}y_n\\
\displaystyle\sum_{n=1}^{N}x_ny_n
\end{pmatrix}
$$

となります．したがって，直線当てはめの最小二乗法の解 $(w_0, w_1)$ が満たす正規方程式は，行列 $X, Y$ を用いて

$$
XX^{\top} \begin{pmatrix} w_0 \\ w_1 \end{pmatrix} = XY^{\top} \qquad (2)
$$

と表されます（ex08notebookA参照）．

同様に，$D$ 個の説明変数から成るデータに式$(1)$を当てはめたい場合，

$$
X = \begin{pmatrix}
1 & 1 & \cdots & 1 \\
x_{1, 1} & x_{2, 1} & \cdots & x_{N, 1}\\
x_{1, 2} & x_{2, 2} & \cdots & x_{N, 2}\\
\vdots & \vdots & \ddots & \vdots \\
x_{1, D} & x_{2, D} & \cdots & x_{N, D}
\end{pmatrix}\quad Y = \begin{pmatrix} y_1 & y_2 & \cdots & y_N\end{pmatrix}
$$

という $(D+1)\times N$ 行列 $X$ と $1\times D$ 行列 $Y$ を作ると，最小二乗法の解を表す正規方程式は

$$
XX^{\top} \begin{pmatrix} w_0 \\ w_1 \\ w_2 \\ \vdots \\ w_D \end{pmatrix} = XY^{\top} \qquad (3)
$$

と表されます（導出その他の説明は「多変量解析及び演習」で行いますので，ここでは気にしないでok）．この式で $D = 1$ とおくと式$(2)$が得られます．つまり，単回帰分析の場合も含めて，データの次元数がいくつであるかによらず，回帰分析の解を表す方程式は行列 $X, Y$ を用いた式$(3)$で表せるわけです．コンピュータを用いて最小二乗法による直線・平面当てはめを行うプログラムでは，実際にこのような行列の計算を行い，式$(3)$の連立方程式を解いて回帰係数を求めるようになっています．

「データ分析」や「多変量解析及び演習」で扱うような問題，さらには，「機械学習I/II」で扱うような問題の多くは，このようにベクトルや行列やを使った式で表されます．
そのため，問題を理解したりその性質を調べたりするために，線形代数がすごく重要となります．またここでは，線形代数が使えると言っても，式が行列で表せるよという単純な話ですが，「多変量解析及び演習」では，行列の固有値や固有ベクトルの話も出てきたりします．

Python では，NumPy という数値計算パッケージを利用することで，行列やベクトルを扱う計算が簡単に書けます（この授業の notebook のほとんどで使ってます）．ゴリゴリ君の最小二乗法のプログラムを実際に書いてみると，次のようになります．

In [ ]:
# データを読み込む
dfGori = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/ex08gorigori.csv', header=0)
N = len(dfGori)

# 行列 X, Y をつくる
X = np.ones((2, N))
X[1, :] = dfGori['気温'].to_numpy()
Y = dfGori['アイス売上数'].to_numpy()
print('X = ')
print(X, X.shape)
print()
print('Y = ', Y, Y.shape)
print()

# 正規方程式の左辺の行列 A と右辺の行列 b を求める
A = X @ X.T  # 行列 A, B に対して A @ B はそれらの積．A.T は A の転置
b = X @ Y.T
print('A = ')
print(A)
print()
print('b = ', b)  # 数学的には 2 x 1 行列だが，一次元配列に格納してるので横に並んで表示される
print()

# 正規方程式の解を求める
w = np.linalg.solve(A, b) # 連立方程式の解を求める関数
print(f'w_0 = {w[0]:.2f}, w_1 = {w[1]:.2f}')  # b = 2.34, a = 2.92

### よだんだよん

この年齡推定の仕組みは，次のようにして作りました．

(1) [IMDB-WIKI dataset](https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/) という研究用画像データセットの中に，Wikipedia から集めた顔画像とその性別，生年月日，撮影年などが含まれたデータ（画像 62,328枚）があったので，これを利用．

(2) 生年月日と撮影年から年齡を算出．顔がちゃんと写ってなかったり年齡の推定値がおかしかったりするデータを削除．顔の領域のみを抜き出す．得られた画像は 38138 枚．この前処理の部分では，こちらのサイトを参考にさせてもらいました：
[顔画像から年齢・性別を推定するためのデータセットIMDB-WIKI](https://qiita.com/yu4u/items/a2410f46669c5f20ee8e)

(3) 得られた画像を $64\times 64$ に縮小してデータ行列 `X` を作成．
`X.shape` は `(38138, 12288)`．

(4) `X` の平均を求めてこれを `X` から引き，特異値分解経由で主成分分析のための固有値・固有ベクトルを算出．次のスペックの Mac で `np.linalg.svd` の実行に23分．
- CPU: Apple M1 Pro
- メモリ: 32GB

(5) 累積寄与率を調べると，2151次元ではじめて99%を超えていたので，2151次元に次元削減した．

(6) そのデータを用いて最小二乗法の解を計算．上記の Mac で `np.linalg.lsesq` の実行に 28 秒．

(7) `X` の平均（12288次元），固有ベクトル（12288x2151），重回帰のパラメータ（2152次元）をファイルに保存．